# Postgres Demo notebook

In this Jupyter notebook we will show an example of adding an additional connector to retrieve metadata from another source.

Notebooks allow us to mix explanatory text and diagrams with functional code (in this case Python). All the code you see is executing in a real environment. Read the [readme](../read-me-first.ipynb) to understand more.

This notebook isn't yet included in our demos by default, but we hope to include more on connectors.

Before running this notebook, you MUST run:
* [configuration notebook](../egeria-server-config.ipynb). 

It's also recommended you read through, and run the following notebooks, to get more background:
* [Building a Data Catalog](../asset-management-labs/building-a-data-catalog.ipynb)
* [Automated Curation](../asset-management-labs/automated-curation.ipynb)


## Infrastructure

These notebooks have been [deployed using Kubernetes](https://egeria-project.org/guides/operations/kubernetes/). We have used a [container](https://quay.io/repository/odpi/egeria-database-connectors) which includes the [Egeria Postgres connector](https://github.com/odpi/egeria-database-connectors) in addition to base Egeria.


First we'll check the existing servers are already started. (They must be configured)



In [6]:
# Standard checks for notebooks - ensure all are running, define standard functions
%run ../common/environment-check.ipynb


Checking OMAG Server Platform availability...
    Core Platform is active
        Checking OMAG Server cocoMDS2
           ... cocoMDS2 is configured
           ... cocoMDS2 is active - ready to begin
        Checking OMAG Server cocoMDS3
           ... cocoMDS3 is configured
           ... cocoMDS3 is active - ready to begin
        Checking OMAG Server cocoMDS5
           ... cocoMDS5 is configured
           ... cocoMDS5 is active - ready to begin
        Checking OMAG Server cocoMDS6
           ... cocoMDS6 is configured
           ... cocoMDS6 is active - ready to begin
    Data Lake Platform is active
        Checking OMAG Server cocoMDS1
           ... cocoMDS1 is configured
           ... cocoMDS1 is active - ready to begin
        Checking OMAG Server cocoMDS4
           ... cocoMDS4 is configured
           ... cocoMDS4 is active - ready to begin
        Checking OMAG Server cocoView1
           ... cocoView1 is configured
           ... cocoView1 is active - ready to begin


Now we are going to setup some standard values that are normally setup elsewhere - some code refactoring of the notebooks is needed to fully integrate.

In [7]:
#language requirement
import os

#admin operations - where/who
adminUserId     = "garygeeke"
devPlatformURL       = os.environ.get('devPlatformURL', 'https://localhost:9445')
devPlatformName      = "Dev Platform"
adminPlatformURL     = devPlatformURL
adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'

# Event infrastructure & cohort membership
eventBusURLroot   = os.environ.get('eventBusURLroot', 'localhost:9092')
jsonContentHeader = {'content-type':'application/json'}
eventBusBody      = {
    "producer": {
        "bootstrap.servers": eventBusURLroot
    },
    "consumer":{
        "bootstrap.servers": eventBusURLroot
    }
}
cocoCohort = "cocoCohort"
devCohort  = "devCohort"
iotCohort  = "iotCohort"

# Default metadata repository options
inMemoryRepositoryOption = "in-memory-repository"
graphRepositoryOption    = "local-graph-repository"

# Pick up which repo type to use from environment if set, otherwise default to inmemory 
metadataRepositoryType   = os.environ.get('repositoryType', inMemoryRepositoryOption)



## Integration Connector

In this notebook we have added an example of using an [Egeria integration connector](https://egeria-project.org/concepts/integration-connector/?h=integration+connector)

![Diagram](https://egeria-project.org/connectors/integration/integration-connectors.svg)

Once started, this connector will retrieve metadata from PostGres. For example this will include:
    - tables
    - columns
    - databases
    
The metadata will then be mapped to [Egeria types](https://egeria-project.org/types/), and pushed into one of the Egeria repositories



## Integration 

We now need to configure the integration server & connector, and point to Postgres from where we will import metadata. More info about this can be found in the [connector readme](https://github.com/odpi/egeria-database-connectors#readme)

![Integration](https://egeria-project.org/concepts/integration-daemon.svg)





## Setup a new metadata server (with repository)

Here we will configure an additional metadata server. This will host both a repository, as well as act as a metadata access point.

See https://egeria-project.org/services/omis/database-integrator/overview/ for more info on the required service to support database integration.

This metadata server will join the 'dev' [cohort](https://egeria-project.org/services/omrs/cohort/?h=cohort) alongside cocoMDSx. 

This will mean it sends events to, and receives events from, other members of the cohort, including the integration server we will add later.

![Coco Cohorts](../images/coco-pharmaceuticals-systems-cohorts.png)

### Required OMASs

Egeria has a grouping of APIs known as the [Open Metadata Access Services](https://egeria-project.org/services/omas/) or OMAS.

The [Data Manager OMAS](https://egeria-project.org/services/omas/data-manager/overview/) is needed to support database integration, so we will enable it in the configuration below.

In [9]:

mdrServerName = 'dbisrv'
mdrServerPlatform = devPlatformURL

# Configure some of the basics (not all) including the required service to support database integration
configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)

# Enable the data-manager service, which is needed to support database integration
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, "data-manager", {})
configureMetadataRepository(adminPlatformURL, adminUserId, mdrServerName, metadataRepositoryType)

# We're going to join this server to the 'dev' cohort
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, devCohort)

# We'll now deploy the created configuration, and start the server
deployServerToPlatform(adminPlatformURL, adminUserId, mdrServerName, devPlatformURL)
activateServerOnPlatform(mdrServerName, "dev platform", devPlatformURL)



   ... configuring the platform the server will run on...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/dbisrv/server-url-root?url=https://lab-dev:9443 )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... configuring the data-manager access service for this server...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/dbisrv/access-services/data-manager )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... configuring the metadata repository...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/dbisrv/local-repository/mode/in-memory-repository )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... configuring the event bus for this server...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/dbisrv/event-bus )
   ...... Response:  {'class': 'VoidRespon

True

## Setup an integration server

We are going to set up a new [integration daemon](https://egeria-project.org/concepts/integration-daemon/) that will run our Postgres connector.

This can run a number of different integration services, which we refer to as our [Open Metadata Integration Services](https://egeria-project.org/services/omis/) (OMIS)

![OMIS](https://egeria-project.org/concepts/integration-daemon-in-action.svg)

This will make use of the [Database Integrator](https://egeria-project.org/services/omis/database-integrator/overview/?h=database+integrator) Open Metadata Integration Service (OMIS). 

In [10]:
# We'll setup a new integration server on the dev platform
intServerName   = "intserv"
intServerPlatform=devPlatformURL

# Config information needed for the postgres connector
pgConnectorUser="pguser"
pgConnectorName="postgresql"
pgConnectorClass="org.odpi.openmetadata.adapters.connectors.integration.postgres.PostgresDatabaseProvider"
integrationServiceURLMarker="database-integrator"

# Authentication and URL for postgres - in a final lab this would be stored in secrets
pgUser="employees"
pgPass=",yDH[4|ETOp<1u*ai+0A{,,q"
pgURL="jdbc:postgresql://1	:5432/employees"

# Setup some of the basics
configurePlatformURL(adminPlatformURL, adminUserId, intServerName, intServerPlatform)
configureDefaultAuditLog(adminPlatformURL, adminUserId, intServerName)




   ... configuring the platform the server will run on...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/intserv/server-url-root?url=https://lab-dev:9443 )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... configuring the default audit log...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/intserv/audit-log-destinations/default )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}


### Connector properties

In the request body we configure properties needed by the connector. The values were set earlier. 

Egeria stores configuration data in encrypted files by default -- and this is pluggable. For example IBM stores configurations along with other cloud data (In cloudant).

In [11]:
# request body needed to setup the postgres connector
integrationServiceOptions={}
connectorConfigs = [
         {
        "class": "IntegrationConnectorConfig",
        "connectorName": "postgresql",
        "connectorUserId": pgConnectorUser,
        "connection":
        {
          "class": "Connection",
          "userId": pgUser,
          "clearPassword": pgPass,
          "connectorType":
          {
            "class": "ConnectorType",
            "connectorProviderClassName": pgConnectorClass
          },
          "endpoint":
          {
            "class": "Endpoint",
            "address": pgURL
          },
          "recognizedConfigurationProperties": [
            "url",
            "ssl"
          ],
          "configurationProperties":
          {
            "url": pgURL,
            "ssl": "false"
          }
        },
        "metadataSourceQualifiedName": pgConnectorName,
        "refreshTimeInterval": "3456",
        "usesBlockingCalls": "false",
        "permittedSynchronization": "FROM_THIRD_PARTY"
      }
    ]

# Setup the integration service, and then deploy the server
configureIntegrationService(adminPlatformURL, adminUserId, intServerName, mdrServerName, 
                                mdrServerPlatform, integrationServiceURLMarker, integrationServiceOptions, 
                                connectorConfigs)

deployServerToPlatform(adminPlatformURL, adminUserId, intServerName, devPlatformURL)



   ... configuring the database-integrator integration service in intserv integration daemon ...
{'class': 'IntegrationServiceRequestBody', 'omagserverPlatformRootURL': 'https://lab-dev:9443', 'omagserverName': 'dbisrv', 'integrationServiceOptions': {}, 'integrationConnectorConfigs': [{'class': 'IntegrationConnectorConfig', 'connectorName': 'postgresql', 'connectorUserId': 'pguser', 'connection': {'class': 'Connection', 'userId': 'employees', 'clearPassword': ',yDH[4|ETOp<1u*ai+0A{,,q', 'connectorType': {'class': 'ConnectorType', 'connectorProviderClassName': 'org.odpi.openmetadata.adapters.connectors.integration.postgres.PostgresDatabaseProvider'}, 'endpoint': {'class': 'Endpoint', 'address': 'jdbc:postgresql://1\t:5432/employees'}, 'recognizedConfigurationProperties': ['url', 'ssl'], 'configurationProperties': {'url': 'jdbc:postgresql://1\t:5432/employees', 'ssl': 'false'}}, 'metadataSourceQualifiedName': 'postgresql', 'refreshTimeInterval': '3456', 'usesBlockingCalls': 'false', 'per

## Starting the integration

We will now start the integration daemon, which will launch our postgres connector, and start pulling in metadata from postgres

Before executing this cell, you may want to [launch the ecosystem UI](https://79c191f6-eu-gb.lb.appdomain.cloud:8091/coco) to see what we have (very little),
for example by using Repository Explorer to view all entities of type RelationalColumn

You could also view the postgres database - For example using a [local install of pgweb](http://localhost:8081).

In [12]:
activateServerOnPlatform(intServerName, "dev platform", devPlatformURL)

                Starting server intserv ...


True

## Checking the metadata that is loaded

On the sample 'employees' database the process of loading the metadata will take around 90s. It will then
enter a poll loop & periodically check for updates

In [20]:
## Experimental area

searchServer="cocoMDSx"
searchPlatformName="dev Platform"
searchPlatformURL=devPlatformURL
searchUserId="garygeeke"
searchString=".*"
searchForAssets(searchServer, searchPlatformName, searchPlatformURL, searchUserId, searchString)

assetConsumerPrintAssets(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, searchUserId, searchString)


No assets found
No assets found


## Summary

We've learnt more about how to pull in metadata from another source using an integration connector.

Please join the community & help us with feedback, ideas, contributions.